#Importing Libraries

In [1]:
!pip install nltk afinn
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import nltk
import csv
import requests
import re
import string
from io import StringIO
from nltk import word_tokenize, pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords, wordnet
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from google.colab import drive
drive.mount('/content/drive')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('sentiwordnet')
nltk.download('averaged_perceptron_tagger')

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#Loading Dataset & Data Cleaning

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data/review_apps_x.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,c0eb381a-9d65-4efd-a775-ae27ccd66df8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya pengguna Twitter yg sekarang berubah menj...,2,23,10.60.0-release.0,9/27/2024 12:55,NaN,NaN,10.60.0-release.0
1,42b1b4c3-799a-4042-b71e-4c410e3616ac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Benar benar tidak nyaman untuk dipakai, memuat...",1,196,10.60.0-release.0,9/27/2024 4:21,NaN,NaN,10.60.0-release.0
2,a3a0d904-9023-49cc-a574-6e56a64fa939,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Search bar ngaco parah gak sesuai sama kata ya...,2,70,10.58.0-release.0,9/22/2024 16:01,NaN,NaN,10.58.0-release.0
3,3ad8dcfc-cae6-4c73-8155-a6b732e09abf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong diperbaiki akun saya sering log-out sen...,1,2122,10.58.0-release.0,9/13/2024 19:59,NaN,NaN,10.58.0-release.0
4,c602f87e-21f5-4e9b-879c-ca776bf19af1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"1. fitur autoscrolling video ilangin aja udah,...",2,119,10.57.0-release.0,9/3/2024 21:45,NaN,NaN,10.57.0-release.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27000 entries, 0 to 26999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   reviewId              27000 non-null  object 
 1   userName              27000 non-null  object 
 2   userImage             27000 non-null  object 
 3   content               27000 non-null  object 
 4   score                 27000 non-null  int64  
 5   thumbsUpCount         27000 non-null  int64  
 6   reviewCreatedVersion  19548 non-null  object 
 7   at                    27000 non-null  object 
 8   replyContent          0 non-null      float64
 9   repliedAt             0 non-null      float64
 10  appVersion            19548 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 2.3+ MB


In [5]:
#kolom dihapus karna data yang dimiliki dari awal hingga akhir 0
df.drop(['replyContent', 'repliedAt'], axis=1, inplace= True)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,c0eb381a-9d65-4efd-a775-ae27ccd66df8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya pengguna Twitter yg sekarang berubah menj...,2,23,10.60.0-release.0,9/27/2024 12:55,10.60.0-release.0
1,42b1b4c3-799a-4042-b71e-4c410e3616ac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Benar benar tidak nyaman untuk dipakai, memuat...",1,196,10.60.0-release.0,9/27/2024 4:21,10.60.0-release.0
2,a3a0d904-9023-49cc-a574-6e56a64fa939,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Search bar ngaco parah gak sesuai sama kata ya...,2,70,10.58.0-release.0,9/22/2024 16:01,10.58.0-release.0
3,3ad8dcfc-cae6-4c73-8155-a6b732e09abf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong diperbaiki akun saya sering log-out sen...,1,2122,10.58.0-release.0,9/13/2024 19:59,10.58.0-release.0
4,c602f87e-21f5-4e9b-879c-ca776bf19af1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"1. fitur autoscrolling video ilangin aja udah,...",2,119,10.57.0-release.0,9/3/2024 21:45,10.57.0-release.0


In [6]:
df.duplicated().sum()

0

In [7]:
df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,7452
at,0
appVersion,7452


In [8]:
df_cleaned = df.dropna()
print(f"Data Isnull : {df_cleaned.isnull().sum().sum()}")
print(f"Data Duplicated : {df_cleaned.duplicated().sum().sum()}")
df_cleaned.info()

Data Isnull : 0
Data Duplicated : 0
<class 'pandas.core.frame.DataFrame'>
Index: 19548 entries, 0 to 26999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              19548 non-null  object
 1   userName              19548 non-null  object
 2   userImage             19548 non-null  object
 3   content               19548 non-null  object
 4   score                 19548 non-null  int64 
 5   thumbsUpCount         19548 non-null  int64 
 6   reviewCreatedVersion  19548 non-null  object
 7   at                    19548 non-null  object
 8   appVersion            19548 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.5+ MB


#Data Preprocessing

In [9]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('indonesian'))
stop_words_en = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Cleaning: remove URLs, special characters, numbers
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Remove mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # Remove hashtag
    text = re.sub(r'RT[\s]', '', text) # Remove RT
    text = re.sub(r'[0-9]+', '', text) # Remove number
    text = re.sub(r'[^\w\s]', '', text) # Remove character besides letters and number
    text = text.replace('\n', ' ') # replace new line with space
    text = text.translate(str.maketrans('', '', string.punctuation)) # Remove all punctuation
    text = text.strip(' ') # Remove character spaces from the left and right of the text

    # Case folding: convert to lowercase
    text = text.lower()

    # Tokenization
    words = word_tokenize(text)

    # Filtering: remove stopwords
    stop_words.update(stop_words_en)
    stop_words.update(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'kalo', 'amp', 'biar', 'bikin', 'bilang', 'iya','yaa','nya','na','sih',
                       'ku',"di","ya","gaa","loh","kah","woi","woii","woy", 'gak', 'ga', 'jir', 'bjir', 'anjir', 'tidak', 'jangan', 'bukan',])
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(words)

# Apply preprocessing to the 'content' column
df_cleaned['cleaned_content'] = df_cleaned['content'].apply(preprocess_text)

# Display the first few rows of cleaned content
df_cleaned.head()

<ipython-input-9-b0139c8b2126>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cleaned_content'] = df_cleaned['content'].apply(preprocess_text)


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion,cleaned_content
0,c0eb381a-9d65-4efd-a775-ae27ccd66df8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya pengguna Twitter yg sekarang berubah menj...,2,23,10.60.0-release.0,9/27/2024 12:55,10.60.0-release.0,pengguna twitter berubah x dr eksis indonesia ...
1,42b1b4c3-799a-4042-b71e-4c410e3616ac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Benar benar tidak nyaman untuk dipakai, memuat...",1,196,10.60.0-release.0,9/27/2024 4:21,10.60.0-release.0,nyaman dipakai memuat medium foto video buka t...
2,a3a0d904-9023-49cc-a574-6e56a64fa939,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Search bar ngaco parah gak sesuai sama kata ya...,2,70,10.58.0-release.0,9/22/2024 16:01,10.58.0-release.0,search bar ngaco parah sesuai dicari dikit mun...
3,3ad8dcfc-cae6-4c73-8155-a6b732e09abf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong diperbaiki akun saya sering log-out sen...,1,2122,10.58.0-release.0,9/13/2024 19:59,10.58.0-release.0,tolong diperbaiki akun logout terkunci capek v...
4,c602f87e-21f5-4e9b-879c-ca776bf19af1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"1. fitur autoscrolling video ilangin aja udah,...",2,119,10.57.0-release.0,9/3/2024 21:45,10.57.0-release.0,fitur autoscrolling video ilangin aja udah ber...


In [10]:
# Initialize VADER
vader_analyzer = SentimentIntensityAnalyzer()

# Initialize AFINN
afinn_analyzer = Afinn()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def vader_sentiment(text):
    vader_score = vader_analyzer.polarity_scores(text)
    return vader_score['compound']

def afinn_sentiment(text):
    return afinn_analyzer.score(text)

def sentiwordnet_sentiment(text):
    words = word_tokenize(text)
    pos_tags = pos_tag(words)

    positive_score = 0
    negative_score = 0
    for word, pos in pos_tags:
        wordnet_pos = get_wordnet_pos(pos)
        if wordnet_pos:
            lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
            senti_synsets = list(swn.senti_synsets(lemma, wordnet_pos))
            if senti_synsets:
                # Take the first synset
                senti_synset = senti_synsets[0]
                positive_score += senti_synset.pos_score()
                negative_score += senti_synset.neg_score()

    # Calculate the final score as the difference between positive and negative
    return positive_score - negative_score

negasi_words = ['tidak', 'bukan', 'kurang', 'belum', 'jangan', 'ga', 'nggak', 'capek', 'cape', 'ngaco', 'heran',
                'parah', 'tak','nih','gabisa','kagak', 'kok', 'aneh', 'jelek','kesel', 'aneh', 'benci', 'jlk', 'gjls', 'payah', 'gabisa',
                'ga bener', 'lemot', 'lama']

def adjust_for_negation(text, sentiment_score):
    words = text.split()
    for word in words:
        if word in negasi_words:
            sentiment_score *= -1  # Return the score if there is negation
    return sentiment_score

def combined_sentiment(text):
   # Calculate score in every lexicon
    vader_score = vader_sentiment(text)
    afinn_score = afinn_sentiment(text)
    sentiwordnet_score = sentiwordnet_sentiment(text)

    # Combine scores by giving more weight to SentiWordNet
    combined_score = (vader_score + afinn_score + 2 * sentiwordnet_score) / 4

    # adjustment for negation
    combined_score = adjust_for_negation(text, combined_score)

    return combined_score

#Labeling

In [11]:
def get_sentiment_label(text):
    combined_score = combined_sentiment(text)

    if combined_score > 0:
        return 'positive'
    elif combined_score < 0:
        return 'negative'
    else:
        return 'neutral'

# Apply function get_polarity to the column 'cleaned_conten' dan create new column 'polarity'
df_cleaned['polarity'] = df_cleaned['cleaned_content'].apply(get_sentiment_label)
df_cleaned.head()

<ipython-input-11-b2d4ef055179>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['polarity'] = df_cleaned['cleaned_content'].apply(get_sentiment_label)


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion,cleaned_content,polarity
0,c0eb381a-9d65-4efd-a775-ae27ccd66df8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya pengguna Twitter yg sekarang berubah menj...,2,23,10.60.0-release.0,9/27/2024 12:55,10.60.0-release.0,pengguna twitter berubah x dr eksis indonesia ...,negative
1,42b1b4c3-799a-4042-b71e-4c410e3616ac,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Benar benar tidak nyaman untuk dipakai, memuat...",1,196,10.60.0-release.0,9/27/2024 4:21,10.60.0-release.0,nyaman dipakai memuat medium foto video buka t...,neutral
2,a3a0d904-9023-49cc-a574-6e56a64fa939,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Search bar ngaco parah gak sesuai sama kata ya...,2,70,10.58.0-release.0,9/22/2024 16:01,10.58.0-release.0,search bar ngaco parah sesuai dicari dikit mun...,negative
3,3ad8dcfc-cae6-4c73-8155-a6b732e09abf,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong diperbaiki akun saya sering log-out sen...,1,2122,10.58.0-release.0,9/13/2024 19:59,10.58.0-release.0,tolong diperbaiki akun logout terkunci capek v...,negative
4,c602f87e-21f5-4e9b-879c-ca776bf19af1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"1. fitur autoscrolling video ilangin aja udah,...",2,119,10.57.0-release.0,9/3/2024 21:45,10.57.0-release.0,fitur autoscrolling video ilangin aja udah ber...,neutral


In [12]:
df_cleaned['polarity'].value_counts()

,count
polarity,
neutral,15451
negative,2161
positive,1936


#Data Augmentation, Data Splitting, dan Ekstraksi Fitur dengan TF-IDF

In [13]:
X = df_cleaned['cleaned_content']
y = df_cleaned['polarity']

# Create oversampling
oversample = RandomOverSampler(sampling_strategy='minority')
X_res, y_res = oversample.fit_resample(X.values.reshape(-1, 1), y)

# Update DataFrame with resampled data
df_balanced = pd.DataFrame({'cleaned_content': X_res.flatten(), 'polarity': y_res})

In [14]:
tfidf_vectorizer = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)

# Fit dan transform data cleaned_content into TF-IDF matrix
X_tfidf = tfidf_vectorizer.fit_transform(df_balanced['cleaned_content'])

# Display the dimension matrix TF-IDF
print(X_tfidf.shape)

(33063, 200)


In [15]:
X = X_tfidf
y = df_balanced['polarity']

# spilt training & testing (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display the dimension set train and test
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (26450, 200)
Shape of X_test: (6613, 200)
Shape of y_train: (26450,)
Shape of y_test: (6613,)


#Modeling

##Model SVM

In [16]:
model = SVC(class_weight='balanced')

# Train model SVM with data training
model.fit(X_train, y_train)

# Predict data test & data train
y_pred_train_svm = model.predict(X_train)
y_pred_test_svm = model.predict(X_test)

# Evaluate accuracy model SVM
accuracy_train_svm = accuracy_score(y_pred_train_svm, y_train)
accuracy_test_svm = accuracy_score(y_pred_test_svm, y_test)

print('SVM - accuracy_train:', accuracy_train_svm)
print('SVM - accuracy_test:', accuracy_test_svm)

print(classification_report(y_test, y_pred_test_svm))

SVM - accuracy_train: 0.943289224952741
SVM - accuracy_test: 0.8935430213216392
              precision    recall  f1-score   support

    negative       0.68      0.47      0.55       432
     neutral       0.86      0.95      0.91      3091
    positive       0.95      0.89      0.92      3090

    accuracy                           0.89      6613
   macro avg       0.83      0.77      0.79      6613
weighted avg       0.89      0.89      0.89      6613



##Model Random Forest Classifier

In [17]:
random_forest = RandomForestClassifier()

# Train model Random Forest with data training
random_forest.fit(X_train, y_train)

# Predict data test & data train
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

# Evaluate accuracy model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

print(classification_report(y_test, y_pred_test_rf))

Random Forest - accuracy_train: 0.970359168241966
Random Forest - accuracy_test: 0.9259035233630727
              precision    recall  f1-score   support

    negative       0.91      0.36      0.52       432
     neutral       0.90      0.98      0.94      3091
    positive       0.95      0.95      0.95      3090

    accuracy                           0.93      6613
   macro avg       0.92      0.76      0.80      6613
weighted avg       0.93      0.93      0.92      6613



##Model Decision Tree Classifier

In [18]:
decision_tree = DecisionTreeClassifier()

# Train model Decision Tree with data training
decision_tree.fit(X_train, y_train)

# Predict data test & data train
y_pred_train_dt = decision_tree.predict(X_train)
y_pred_test_dt = decision_tree.predict(X_test)

# Evaluate accuracy model Random Forest
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

print(classification_report(y_test, y_pred_test_dt))

Decision Tree - accuracy_train: 0.970359168241966
Decision Tree - accuracy_test: 0.8847724179646151
              precision    recall  f1-score   support

    negative       0.56      0.41      0.48       432
     neutral       0.90      0.88      0.89      3091
    positive       0.90      0.95      0.93      3090

    accuracy                           0.88      6613
   macro avg       0.79      0.75      0.76      6613
weighted avg       0.88      0.88      0.88      6613

